In [20]:
#Sequential model // doesnt work, refer to sequential vs. functional api
#Functional api's handle multiple inputs better, sequential cannot create model with multiple inputs
import keras
import tensorflow
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import Conv1D
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Conv3D
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Concatenate
from keras.layers import Input
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras.models import load_model


load = np.load('data/RAVDESS.npz')
audio = load['audio']
video = load['video']
labels = load['labels']

#video pre-process
fraction = 5
frames = int(80/fraction)
shape = (128,128,frames,1)

reshape_videos = []

for i in video:
    single_video = []
    for j in range( i.shape[0] ): #HEREi.shape[0]
        if(  j % fraction == 0 ):
            single_video.append( i[j] )
    reshape_videos.append( single_video )

video = np.array(reshape_videos)
del reshape_videos

video_input = (128, 128, frames, 1)
video_shape = (video.shape[0], 128, 128, frames, 1)
video = np.reshape(video, video_shape)
####################


#audio pre-process
num_rows = 105211
num_columns = 1
num_channels = 1

audio_input = (num_rows, num_columns)
audio_shape = (audio.shape[0], num_rows, num_columns)
audio = np.reshape(audio, audio_shape)
###################


v_train, v_test, a_train, a_test, l_train, l_test = train_test_split(video, 
                                                                     audio, 
                                                                     labels,
                                                                     test_size=0.166,
                                                                     random_state=42) #size=0.2

print(v_train.shape)
print(l_train.shape)
print(a_train.shape)


def audio_cnn_1d(sample_shape):
    #Layer 1
    input_ = Input(shape=sample_shape)
    conv1d = Conv1D(padding='same', filters=25, kernel_size=64, strides = 8, input_shape=sample_shape)(input_)
    relu = LeakyReLU(alpha=0.2)(conv1d)
    
    #Layer 2
    conv1d = Conv1D(padding='same', filters=25, kernel_size=128, strides = 8)(relu)
    batch = BatchNormalization()(conv1d)
    relu = LeakyReLU(alpha=0.2)(batch)
    
    #Layer 3
    conv1d = Conv1D(padding='same', filters=25, kernel_size=256, strides = 8)(relu)
    batch = BatchNormalization()(conv1d)
    relu = LeakyReLU(alpha=0.2)(batch)
    
    #Layer 4
    conv1d = Conv1D(padding='same', filters=25, kernel_size=512, strides = 4)(relu)
    batch = BatchNormalization()(conv1d)
    relu = LeakyReLU(alpha=0.2)(batch)
    
    #Layer 5
    conv1d = Conv1D(padding='same', filters=25, kernel_size=1024, strides = 4)(relu)
    batch = BatchNormalization()(conv1d)
    relu = LeakyReLU(alpha=0.2)(relu)
    
    #layer 6
    output = Flatten()(relu)
    
    model = Model(inputs=[input_], outputs=[output] )
    return model

#video neural network
def video_cnn_3d(sample_shape):
    input_ = Input(shape=sample_shape)
    #layer 1
    conv3d = Conv3D(64, kernel_size=(3,3,3), strides=(2,2,1), input_shape=sample_shape)(input_)
    relu = LeakyReLU(alpha=0.2)(conv3d)
    
    #layer 2
    conv3d = Conv3D(128, kernel_size=(3,3,3), strides=(2,2,1), padding='SAME')(relu)
    batch = BatchNormalization()(conv3d)
    relu = LeakyReLU(alpha=0.2)(batch)
    
    #layer 3
    conv3d = Conv3D(256, kernel_size=(3,3,3), strides=(2,2,1), padding='SAME')(relu)
    batch = BatchNormalization()(conv3d)
    relu = LeakyReLU(alpha=0.2)(batch)

    #layer 4
    conv3d = Conv3D(512, kernel_size=(3,3,3), strides=(2,2,2), padding='SAME')(relu)
    batch = BatchNormalization()(conv3d)
    relu = LeakyReLU(alpha=0.2)(batch)

    #layer 5
    conv3d = Conv3D(1024, kernel_size=(3,3,3), strides=(2,2,2), padding='SAME')(relu)
    batch = BatchNormalization()(conv3d)
    relu = LeakyReLU(alpha=0.2)(batch)

    #layer 6
    conv3d = Conv3D(2048, kernel_size=(3,3,3), strides=(2,2,2), padding='SAME')(relu)
    batch = BatchNormalization()(conv3d)
    relu = LeakyReLU(alpha=0.2)(batch)

    #layer 7
    output = Flatten()(relu)
    model = Model(inputs=[input_], outputs=[output] )
    return model



audio_model = audio_cnn_1d(audio_input)
audio_model.summary()

video_model = video_cnn_3d(video_input)
video_model.summary()


left_input = Input(audio_input)
right_input = Input(video_input)

l_encoded = audio_model(left_input)
r_encoded = video_model(right_input)

#fusion layer
fusion = Concatenate()([l_encoded,r_encoded])
batch = BatchNormalization()(fusion)
prediction = Dense(1,activation='softmax')(batch)
twin_net = Model([left_input,right_input],prediction)


twin_net.compile(loss='binary_crossentropy',
        optimizer=optimizers.Adam(lr=0.00001),metrics=['accuracy'])

twin_net.summary()

twin_net.fit([a_train,v_train], l_train,
        batch_size=30,
        epochs=30,
        verbose=1,
        validation_split=0.166)

twin_net.save('models/twin_net_softmax')


#test
model = load_model('models/twin_net_softmax')


score = model.evaluate([a_train, v_train], l_train, verbose=0)
print("Training Accuracy: ", score)

score = model.evaluate([a_test, v_test], l_test, verbose=1)
print("Testing Accuracy: ", score)





(320, 128, 128, 16, 1)
(320,)
(320, 105211, 1)
Model: "model_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_69 (InputLayer)        [(None, 105211, 1)]       0         
_________________________________________________________________
conv1d_85 (Conv1D)           (None, 13152, 25)         1625      
_________________________________________________________________
leaky_re_lu_187 (LeakyReLU)  (None, 13152, 25)         0         
_________________________________________________________________
conv1d_86 (Conv1D)           (None, 1644, 25)          80025     
_________________________________________________________________
batch_normalization_168 (Bat (None, 1644, 25)          100       
_________________________________________________________________
leaky_re_lu_188 (LeakyReLU)  (None, 1644, 25)          0         
_________________________________________________________________
conv1d_87 (

KeyboardInterrupt: 

In [ ]:
(320, 128, 128, 16, 1)
(320,)
(320, 105211, 1)
Model: "model_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_61 (InputLayer)        [(None, 105211, 1)]       0         
_________________________________________________________________
conv1d_75 (Conv1D)           (None, 13152, 25)         1625      
_________________________________________________________________
leaky_re_lu_165 (LeakyReLU)  (None, 13152, 25)         0         
_________________________________________________________________
conv1d_76 (Conv1D)           (None, 1644, 25)          80025     
_________________________________________________________________
batch_normalization_148 (Bat (None, 1644, 25)          100       
_________________________________________________________________
leaky_re_lu_166 (LeakyReLU)  (None, 1644, 25)          0         
_________________________________________________________________
conv1d_77 (Conv1D)           (None, 206, 25)           160025    
_________________________________________________________________
batch_normalization_149 (Bat (None, 206, 25)           100       
_________________________________________________________________
leaky_re_lu_167 (LeakyReLU)  (None, 206, 25)           0         
_________________________________________________________________
conv1d_78 (Conv1D)           (None, 52, 25)            320025    
_________________________________________________________________
batch_normalization_150 (Bat (None, 52, 25)            100       
_________________________________________________________________
leaky_re_lu_168 (LeakyReLU)  (None, 52, 25)            0         
_________________________________________________________________
leaky_re_lu_169 (LeakyReLU)  (None, 52, 25)            0         
_________________________________________________________________
flatten_30 (Flatten)         (None, 1300)              0         
=================================================================
Total params: 562,000
Trainable params: 561,850
Non-trainable params: 150
_________________________________________________________________
Model: "model_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_62 (InputLayer)        [(None, 128, 128, 16, 1)] 0         
_________________________________________________________________
conv3d_90 (Conv3D)           (None, 63, 63, 14, 64)    1792      
_________________________________________________________________
leaky_re_lu_170 (LeakyReLU)  (None, 63, 63, 14, 64)    0         
_________________________________________________________________
conv3d_91 (Conv3D)           (None, 32, 32, 14, 128)   221312    
_________________________________________________________________
batch_normalization_152 (Bat (None, 32, 32, 14, 128)   512       
_________________________________________________________________
leaky_re_lu_171 (LeakyReLU)  (None, 32, 32, 14, 128)   0         
_________________________________________________________________
conv3d_92 (Conv3D)           (None, 16, 16, 14, 256)   884992    
_________________________________________________________________
batch_normalization_153 (Bat (None, 16, 16, 14, 256)   1024      
_________________________________________________________________
leaky_re_lu_172 (LeakyReLU)  (None, 16, 16, 14, 256)   0         
_________________________________________________________________
conv3d_93 (Conv3D)           (None, 8, 8, 7, 512)      3539456   
_________________________________________________________________
batch_normalization_154 (Bat (None, 8, 8, 7, 512)      2048      
_________________________________________________________________
leaky_re_lu_173 (LeakyReLU)  (None, 8, 8, 7, 512)      0         
_________________________________________________________________
conv3d_94 (Conv3D)           (None, 4, 4, 4, 1024)     14156800  
_________________________________________________________________
batch_normalization_155 (Bat (None, 4, 4, 4, 1024)     4096      
_________________________________________________________________
leaky_re_lu_174 (LeakyReLU)  (None, 4, 4, 4, 1024)     0         
_________________________________________________________________
conv3d_95 (Conv3D)           (None, 2, 2, 2, 2048)     56625152  
_________________________________________________________________
batch_normalization_156 (Bat (None, 2, 2, 2, 2048)     8192      
_________________________________________________________________
leaky_re_lu_175 (LeakyReLU)  (None, 2, 2, 2, 2048)     0         
_________________________________________________________________
flatten_31 (Flatten)         (None, 16384)             0         
=================================================================
Total params: 75,445,376
Trainable params: 75,437,440
Non-trainable params: 7,936
_________________________________________________________________
Model: "model_41"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_63 (InputLayer)           [(None, 105211, 1)]  0                                            
__________________________________________________________________________________________________
input_64 (InputLayer)           [(None, 128, 128, 16 0                                            
__________________________________________________________________________________________________
model_39 (Functional)           (None, 1300)         562000      input_63[0][0]                   
__________________________________________________________________________________________________
model_40 (Functional)           (None, 16384)        75445376    input_64[0][0]                   
__________________________________________________________________________________________________
concatenate_14 (Concatenate)    (None, 17684)        0           model_39[0][0]                   
                                                                 model_40[0][0]                   
__________________________________________________________________________________________________
batch_normalization_157 (BatchN (None, 17684)        70736       concatenate_14[0][0]             
__________________________________________________________________________________________________
dense_32 (Dense)                (None, 1)            17685       batch_normalization_157[0][0]    
==================================================================================================
Total params: 76,095,797
Trainable params: 76,052,343
Non-trainable params: 43,454
__________________________________________________________________________________________________
Epoch 1/30
9/9 [==============================] - 5s 370ms/step - loss: 0.7571 - accuracy: 0.6567 - val_loss: 0.6352 - val_accuracy: 0.6667
Epoch 2/30
9/9 [==============================] - 3s 330ms/step - loss: 0.1549 - accuracy: 0.9234 - val_loss: 0.6179 - val_accuracy: 0.6111
Epoch 3/30
9/9 [==============================] - 3s 328ms/step - loss: 0.0381 - accuracy: 0.9826 - val_loss: 0.6149 - val_accuracy: 0.5741
Epoch 4/30
9/9 [==============================] - 3s 329ms/step - loss: 0.0125 - accuracy: 0.9984 - val_loss: 0.6148 - val_accuracy: 0.5741
Epoch 5/30
9/9 [==============================] - 3s 329ms/step - loss: 0.0033 - accuracy: 0.9978 - val_loss: 0.6057 - val_accuracy: 0.5741
Epoch 6/30
9/9 [==============================] - 3s 328ms/step - loss: 0.0085 - accuracy: 0.9952 - val_loss: 0.5948 - val_accuracy: 0.5741
Epoch 7/30
9/9 [==============================] - 3s 335ms/step - loss: 0.0105 - accuracy: 0.9902 - val_loss: 0.5793 - val_accuracy: 0.6111
Epoch 8/30
9/9 [==============================] - 3s 346ms/step - loss: 8.3730e-04 - accuracy: 1.0000 - val_loss: 0.5642 - val_accuracy: 0.6481
Epoch 9/30
9/9 [==============================] - 3s 353ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.5407 - val_accuracy: 0.6852
Epoch 10/30
9/9 [==============================] - 3s 343ms/step - loss: 2.2556e-04 - accuracy: 1.0000 - val_loss: 0.5148 - val_accuracy: 0.7037
Epoch 11/30
9/9 [==============================] - 3s 335ms/step - loss: 0.0025 - accuracy: 0.9984 - val_loss: 0.4896 - val_accuracy: 0.7407
Epoch 12/30
9/9 [==============================] - 3s 333ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.4658 - val_accuracy: 0.7593
Epoch 13/30
9/9 [==============================] - 3s 332ms/step - loss: 2.8409e-04 - accuracy: 1.0000 - val_loss: 0.4393 - val_accuracy: 0.8148
Epoch 14/30
9/9 [==============================] - 3s 338ms/step - loss: 1.3621e-04 - accuracy: 1.0000 - val_loss: 0.4120 - val_accuracy: 0.8148
Epoch 15/30
9/9 [==============================] - 3s 347ms/step - loss: 4.5485e-04 - accuracy: 1.0000 - val_loss: 0.3815 - val_accuracy: 0.8333
Epoch 16/30
9/9 [==============================] - 3s 347ms/step - loss: 2.4808e-04 - accuracy: 1.0000 - val_loss: 0.3506 - val_accuracy: 0.8889
Epoch 17/30
9/9 [==============================] - 3s 341ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.3259 - val_accuracy: 0.8889
Epoch 18/30
9/9 [==============================] - 3s 335ms/step - loss: 1.0890e-04 - accuracy: 1.0000 - val_loss: 0.3087 - val_accuracy: 0.8889
Epoch 19/30
9/9 [==============================] - 3s 329ms/step - loss: 2.1318e-04 - accuracy: 1.0000 - val_loss: 0.2900 - val_accuracy: 0.9259
Epoch 20/30
9/9 [==============================] - 3s 338ms/step - loss: 2.7181e-04 - accuracy: 1.0000 - val_loss: 0.2722 - val_accuracy: 0.9259
Epoch 21/30
9/9 [==============================] - 3s 345ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.2516 - val_accuracy: 0.9444
Epoch 22/30
9/9 [==============================] - 3s 346ms/step - loss: 6.1500e-04 - accuracy: 1.0000 - val_loss: 0.2371 - val_accuracy: 0.9444
Epoch 23/30
9/9 [==============================] - 3s 353ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.2295 - val_accuracy: 0.9259
Epoch 24/30
9/9 [==============================] - 3s 336ms/step - loss: 4.1407e-04 - accuracy: 1.0000 - val_loss: 0.2172 - val_accuracy: 0.9259
Epoch 25/30
9/9 [==============================] - 3s 334ms/step - loss: 2.4711e-04 - accuracy: 1.0000 - val_loss: 0.2254 - val_accuracy: 0.9259
Epoch 26/30
9/9 [==============================] - 3s 327ms/step - loss: 3.4520e-04 - accuracy: 1.0000 - val_loss: 0.2332 - val_accuracy: 0.9259
Epoch 27/30
9/9 [==============================] - 3s 334ms/step - loss: 1.4777e-04 - accuracy: 1.0000 - val_loss: 0.2393 - val_accuracy: 0.9259
Epoch 28/30
9/9 [==============================] - 3s 344ms/step - loss: 2.9401e-04 - accuracy: 1.0000 - val_loss: 0.2433 - val_accuracy: 0.9259
Epoch 29/30
9/9 [==============================] - 3s 343ms/step - loss: 1.7767e-04 - accuracy: 1.0000 - val_loss: 0.2473 - val_accuracy: 0.9259
Epoch 30/30
9/9 [==============================] - 3s 348ms/step - loss: 9.3719e-05 - accuracy: 1.0000 - val_loss: 0.2535 - val_accuracy: 0.9259
INFO:tensorflow:Assets written to: models/twin_net/assets
Training Accuracy:  [0.04587867110967636, 0.987500011920929]
2/2 [==============================] - 0s 93ms/step - loss: 0.1711 - accuracy: 0.9375
Testing Accuracy:  [0.17109230160713196, 0.9375]